In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files


# STEP 1: PREPARE DATA
df_public = pd.read_csv("../source_data/SOURCE-HDB-2019-03-FilteredTowns _1990-All-Flat-Types.csv", parse_dates=['month'])
df_public['floor_area_sqft'] = df_public['floor_area_sqm'] * 10.7639

common_start_date = pd.Timestamp('2019-03-01')

df_public['month'] = pd.to_datetime(df_public['month'])
public_start_date = df_public['month'].min()
df_public['months_since_common_start'] = ((df_public['month'] - common_start_date) / np.timedelta64(1, 'M')).astype(int)

# print(f"df_public {df_public}")


# Define the projects
projects_to_calculate = [
    # CENTRAL
    ('CANTONMENT RD', 2011),
    ('KLANG LANE', [2002,2003]),

    # BUKIT MERAH
    ('TELOK BLANGAH HTS', 2003),
    ('Kim Tian Rd', 2013),
    ('HENDERSON RD', 2019),
    ('JLN MEMBINA', [2008, 2009]),
    ('HAVELOCK RD', 2013),
    ('REDHILL RD', [2005, 2007]),
    ('TELOK BLANGAH ST 31', 2013),
    ('TELOK BLANGAH ST 31', 2018),
    ('TELOK BLANGAH HTS', 2017),
    ('TELOK BLANGAH CRES', [1999, 2002]),
    ('BOON TIONG RD', 2001),
    ('BOON TIONG RD', 2005),
    ('BOON TIONG RD', 2016),
    ('CANTONMENT CL', [2002, 2003]),
    ('DEPOT RD', [2000, 2002]),
    ('DEPOT RD', 2006),
    ('DEPOT RD', 2017),
    ('KIM TIAN PL', 2001),
    ('LENGKOK BAHRU', 2001),

    # QUEENSTOWN
    ('DAWSON RD', 2016),
    ('C\'WEALTH DR', 2015),
    ('DOVER CRES', 2012),
    ('GHIM MOH LINK', 2013),
    ('GHIM MOH LINK', 2018),
    ('HOLLAND DR', 2012),
    ('STIRLING RD', 2000),
    ('STRATHMORE AVE', [2000, 2002]),
    ('STRATHMORE AVE', 2006),
    ('STRATHMORE AVE', 2011),
    ('STRATHMORE AVE', 2020),
    ('TANGLIN HALT RD', 2008),

    # KALLANG/WHAMPOA
    ('UPP BOON KENG RD', 2017),
    ('BENDEMEER RD', 2012),
    ('JELLICOE RD', 2003),
    ('MCNAIR RD', 2012),
    ('ST. GEORGE\'S LANE', 1996),
    ('WHAMPOA DR', 1998),
    ('WHAMPOA RD', [1994, 1995]),

    # CLEMENTI
    ('CLEMENTI AVE 4', [2014, 2017]),
    ('CLEMENTI AVE 1', 2013),
    ('CLEMENTI AVE 3', 2006),
    ('CLEMENTI AVE 3', 2012),
    ('CLEMENTI AVE 3', 2018),

    # JURONG EAST
    ('TEBAN GDNS RD', 2013),

    # GEYLANG
    ('JLN TIGA', 2005),
    ('EUNOS CRES', 2006),

    # HOUGANG
    ('BUANGKOK CRES', [2003, 2004]),
    ('BUANGKOK CRES', 2018),
    ('BUANGKOK LINK', 2004),
    ('HOUGANG AVE 1', [1997, 1998]),
    ('HOUGANG AVE 10', [1991, 1992]),
    ('HOUGANG AVE 4', [1997, 1998]),
    ('HOUGANG AVE 5', 1992),
    ('HOUGANG AVE 8', [1991, 1992]),
    ('HOUGANG AVE 9', [1998, 1999]),
    ('HOUGANG CTRL', [1992, 1997]),
    ('HOUGANG ST 31', [1999, 2000]),
    ('HOUGANG ST 51', 1997),
    ('HOUGANG ST 91', [1997, 1998, 1999]),
    ('HOUGANG ST 92', 1998),
    ('HOUGANG AVE 5', 1992),
    ('HOUGANG AVE 5', 1992),
    ('HOUGANG AVE 5', 1992),
    ('UPP SERANGOON CRES', 2014),
    ('UPP SERANGOON VIEW', 2016),
    ('UPP SERANGOON CRES', [2014, 2015, 2017]),

    # BISHAN
    ('BISHAN ST 24', 2011),

    # ANG MO KIO
    ('ANG MO KIO AVE 1', 2012),
    ('ANG MO KIO AVE 6', 2003),
    ('ANG MO KIO ST 21', 2018),
    ('ANG MO KIO ST 31', 2006),
    ('ANG MO KIO ST 44', 2018),
    ('ANG MO KIO ST 52', [2002, 2003]),
    ('ANG MO KIO ST 52', 2011),

    # JURONG WEST
    ('BOON LAY AVE', [2015, 2018]),
    ('BOON LAY DR', [2002, 2015]),
    ('CORPORATION DR', 1997),
    ('HO CHING RD', 1999),
    ('JURONG WEST AVE 3', [2001, 2002]),
    ('JURONG WEST CTRL 1', 2000),
    ('JURONG WEST CTRL 3', 2017),
    ('JURONG WEST ST 64', 2008),
    ('JURONG WEST ST 93', 2008),
    ('KANG CHING RD', 2012),
    ('YUAN CHING RD', 2014),
    ('YUNG KUANG RD', [2015, 2018]),

    # SENGKANG
    ('ANCHORVALE CRES', [2015, 2018]),
    ('ANCHORVALE LINK', 2015),
    ('ANCHORVALE RD', [2001, 2002, 2003]),
    ('ANCHORVALE RD', 2015),
    ('ANCHORVALE ST', 2015),
    ('COMPASSVALE BOW', 2017),
    ('COMPASSVALE CRES', [2001, 2002]),
    ('COMPASSVALE CRES', [2016, 2017]),
    ('COMPASSVALE DR', [1999, 2000, 2001]),
    ('COMPASSVALE DR', [2012, 2017]),
    ('COMPASSVALE LINK', 2006),
    ('COMPASSVALE ST', [2000, 2001, 2002]),
    ('COMPASSVALE WALK', [1999, 2000]),
    ('FERNVALE LANE', 2004),
    ('FERNVALE LINK', [2015, 2016]),
    ('FERNVALE RD', [2004, 2005]),
    ('FERNVALE RD', 2013),
    ('FERNVALE ST', 2016),
    ('RIVERVALE CRES', 2017),
    ('SENGKANG EAST AVE', 2015),
    ('SENGKANG WEST WAY', [2014, 2015, 2016]),

    # PUNGGOL
    ('PUNGGOL DR', 2005),
    ('PUNGGOL DR', [2013, 2016]),
    ('EDGEDALE PLAINS', [2003, 2004]),
    ('EDGEDALE PLAINS', [2017, 2018]),
    ('EDGEFIELD PLAINS', 2016),
    ('PUNGGOL EAST', 2018),
    ('PUNGGOL FIELD', [2013, 2014, 2016]),
    ('PUNGGOL PL', [2012, 2013, 2014]),
    ('PUNGGOL WALK', [2014, 2015, 2016]),
    ('PUNGGOL WAY', [2015, 2016, 2017]),
    ('SUMANG LANE', 2018),
    ('SUMANG LINK', 2015),
    ('SUMANG WALK', [2017, 2019, 2020]),

    # TOA PAYOH
    ('JOO SENG RD', 1995),
    ('LOR 1 TOA PAYOH', 1995),
    ('LOR 1 TOA PAYOH', 1999),
    ('LOR 1A TOA PAYOH', 2012),
    ('LOR 2 TOA PAYOH', [1999, 2001, 2002]),
    ('LOR 4 TOA PAYOH', [1996, 1997, 1999]),
    ('LOR 7 TOA PAYOH', 1998),
    ('TOA PAYOH CTRL', 2009),
]

gradients = []

def filter_project(street_name, years):
    if isinstance(years, list):
        return df_public[(df_public['street_name'].str.upper() == street_name.upper()) &
                    (df_public['lease_commence_date'].between(years[0]-5, years[-1]+5))].copy()
    else:
        return df_public[(df_public['street_name'].str.upper() == street_name.upper()) &
                    (df_public['lease_commence_date'].between(years-5, years+5))].copy()


for street_name, years in projects_to_calculate:
    project_data = filter_project(street_name, years)

    if project_data.empty:
        print(f"No data available for {street_name} ({years}).")
        continue

    num_data_points = len(project_data)
    if num_data_points < 4:
      continue

    # Calculate the Unit Price (PSF) for each entry
    project_data['unit_price_psf'] = project_data['resale_price'] / project_data['floor_area_sqft']

    X = project_data[['months_since_common_start']].values
    y = project_data['unit_price_psf'].values
    model = LinearRegression().fit(X, y)
    project_data['predicted_psf'] = model.predict(X)

    # Get start and end dates within the dataset
    start_date = project_data['month'].min()
    end_date = project_data['month'].max()

    # Calculate the number of months between start and end dates
    months_between = ((end_date - start_date) / np.timedelta64(1, 'M'))

    # Predict Unit Price PSF at the start and end of the regression time period
    predicted_psf_start = model.predict(np.array([[project_data['months_since_common_start'].min()]]))[0]
    predicted_psf_end = model.predict(np.array([[project_data['months_since_common_start'].max()]]))[0]

    base_price = predicted_psf_start

    # Calculate the gradient of Unit Price (PSF) over time
    monthly_gradient = model.coef_[0]
    gradient_per_year = monthly_gradient * 12
    gradient_percentage = (gradient_per_year / base_price) * 100
    town = project_data['town'].mode()[0]
    gradients.append((town, street_name, years, gradient_per_year, gradient_percentage, predicted_psf_start, predicted_psf_end, start_date, end_date, months_between, num_data_points))


    print(f"Gradient for {street_name}: gradient_per_year {gradient_per_year:.4f}, gradient_percentage {gradient_percentage:.2f}%")
    print(f"Predicted PSF at start: {predicted_psf_start:.2f}, at end: {predicted_psf_end:.2f}, from {start_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}, over {months_between} months. {num_data_points} data points")


# Convert the list of gradients to a DataFrame
gradients_df = pd.DataFrame(gradients, columns=['Town', 'HDB Street Name', 'TOP Year(s)', 'Gradient ($PSF per year)', 'Gradient % ($PSF per year)', 'Predicted PSF Start', 'Predicted PSF End', 'Start Date', 'End Date', 'Months Between', 'No of Data Points'])
print(f"gradients_df: {gradients_df}")
gradients_df.to_csv('gradients_df.csv', index=False)
files.download('gradients_df.csv')





Gradient for CANTONMENT RD: gradient_per_year 66.1629, gradient_percentage 7.61%
Predicted PSF at start: 869.45, at end: 1189.24, from 2019-03 to 2024-03, over 58.935483870967744 months. 320 data points
Gradient for KLANG LANE: gradient_per_year 33.3733, gradient_percentage 6.39%
Predicted PSF at start: 522.28, at end: 680.81, from 2019-03 to 2024-01, over 57.0 months. 60 data points
Gradient for TELOK BLANGAH HTS: gradient_per_year 37.2048, gradient_percentage 6.41%
Predicted PSF at start: 580.38, at end: 760.20, from 2019-03 to 2024-03, over 58.935483870967744 months. 78 data points
Gradient for Kim Tian Rd: gradient_per_year 39.4536, gradient_percentage 5.19%
Predicted PSF at start: 760.91, at end: 951.60, from 2019-03 to 2024-03, over 58.935483870967744 months. 168 data points
Gradient for HENDERSON RD: gradient_per_year 51.8207, gradient_percentage 5.94%
Predicted PSF at start: 873.04, at end: 1054.42, from 2020-08 to 2024-03, over 42.193548387096776 months. 102 data points
Gradie

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Plot graph for selected projects
# TODO:  save the URLs to dataframe in GDrive and allow collaborators to access the GDrive folder
# & download HTML file locally to interact with it

# !pip install -U -q PyDrive  # Only if saving files to GDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# def upload_file_to_drive(file_path, parent_folder_id):
#     """Uploads a file to Google Drive and returns its shareable link."""
#     file_name = os.path.basename(file_path)
#     file = drive.CreateFile({'title': file_name,
#                              'parents': [{'id': parent_folder_id}]})
#     file.SetContentFile(file_path)
#     file.Upload()
#     file.InsertPermission({
#         'type': 'anyone',
#         'value': 'anyone',
#         'role': 'reader'
#     })
#     return file['alternateLink']

# parent_folder_id = '1ft1VEoDD8CEls1W_HmbPTqg_QvU2dlNR'  # folder ID of "Python-generated Graphs" in Drive

# drive.mount('/content/drive')
# folder_path = '/content/drive/My Drive/1 Projects/ML Sing Property Prices/Python-generated Graphs'

# import os
# os.makedirs(folder_path, exist_ok=True)  # Ensure the folder exists


projects_to_plot = [
    # CENTRAL
    ('CANTONMENT RD', 2011),
    ('KLANG LANE', [2002, 2003]),

    # # BUKIT MERAH
    # ('DEPOT RD', [2000, 2002]),
    # ('DEPOT RD', 2006),

    # QUEENSTOWN
    ('LENGKOK BAHRU', 2001),

    # # ANG MO KIO
    # ('ANG MO KIO ST 21', 2018),

    # # KALLANG/WHAMPOA
    # ('WHAMPOA DR', 1998),
    # ('WHAMPOA RD', [1994, 1995]),

    # # HOUGANG
    # ('BUANGKOK CRES', [2003, 2004]),
    # ('BUANGKOK CRES', 2018),
    # ('BUANGKOK LINK', 2004),
]

for street_name, years in projects_to_plot:
    project_data = filter_project(street_name, years)
    if project_data.empty:
        print(f"No data available for {street_name} ({years}).")
        continue

    project_data['unit_price_psf'] = project_data['resale_price'] / project_data['floor_area_sqft']

    X = project_data[['months_since_common_start']].values
    y = project_data['unit_price_psf'].values
    model = LinearRegression().fit(X, y)
    project_data['predicted_psf'] = model.predict(X)

    predicted_psf_start = model.predict(np.array([[project_data['months_since_common_start'].min()]]))[0]
    predicted_psf_end = model.predict(np.array([[project_data['months_since_common_start'].max()]]))[0]

    hover_text = ['Date: {}<br>Unit Price PSF: ${:.2f}'.format(date.strftime('%Y-%m-%d'), psf)
              for date, psf in zip(project_data['month'], project_data['predicted_psf'])]

    fig = px.scatter(project_data, x='month', y='unit_price_psf',
                    hover_data=['resale_price', 'floor_area_sqft', 'storey_range'],
                    labels={'unit_price_psf': 'Unit Price (PSF)', 'month': 'Time'},
                    title=f"Unit Price (PSF) Over Time for {street_name} ({years})")


    fig.add_trace(go.Scatter(x=project_data['month'], y=project_data['predicted_psf'],
                            mode='lines', name='Regression Line',
                            hoverinfo='text',
                            text=hover_text))


    fig.show()


    #!#$%^&*^*REFACTOR
    base_year = project_data['month'].dt.year.min()
    base_year_data = project_data[project_data['month'].dt.year == base_year].copy()
    base_year_data['unit_price_psf'] = (base_year_data['resale_price'] / base_year_data['floor_area_sqft'])
    base_price = base_year_data['unit_price_psf'].mean()

    # Calculate the gradient of Unit Price (PSF) over time
    monthly_gradient = model.coef_[0]
    gradient_per_year = monthly_gradient * 12
    gradient_percentage = (gradient_per_year / base_price) * 100

    #TISH DEBUG
    print(f"TISH DEBUG Comparing base_price {base_price} to Predicted PSF at start: {predicted_psf_start:.2f}")
    print(f"TISH DEBUG gradient_percentage divided by base_price {gradient_percentage} to gradient_percentage divided by Predicted PSF start: {(gradient_per_year / predicted_psf_start) * 100}")

    start_date = project_data['month'].min()
    end_date = project_data['month'].max()
    months_between = ((end_date - start_date) / np.timedelta64(1, 'M'))
    num_data_points = len(project_data)

    print(f"Gradient for {street_name}: gradient_per_year {gradient_per_year:.4f}, gradient_percentage {gradient_percentage:.2f}%")
    print(f"Predicted PSF at start: {predicted_psf_start:.2f}, at end: {predicted_psf_end:.2f}, from {start_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}, over {months_between} months. {num_data_points} data points")

    year_str = str(years[0]) if isinstance(years, list) else str(years)


    # file_name = f"{street_name.replace(' ', '_')}_{year_str}_Price_Trend.html"  # Replace spaces with underscores
    # link = upload_file_to_drive(file_name, parent_folder_id)
    # print(link)


    # full_path = os.path.join(folder_path, file_name)  # Save to Google Drive

    # fig.write_html(full_path)
    print(f"Saved: {link}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TISH DEBUG Comparing base_price 941.4267847819332 to Predicted PSF at start: 869.45
TISH DEBUG gradient_percentage divided by base_price 7.027943424787976 to gradient_percentage divided by Predicted PSF start: 7.60970330576842
Gradient for CANTONMENT RD: gradient_per_year 66.1629, gradient_percentage 7.03%
Predicted PSF at start: 869.45, at end: 1189.24, from 2019-03 to 2024-03, over 58.935483870967744 months. 320 data points
Saved: /content/drive/My Drive/1 Projects/ML Sing Property Prices/Python-generated Graphs/CANTONMENT_RD_2011_Price_Trend.html


TISH DEBUG Comparing base_price 560.5479383786212 to Predicted PSF at start: 522.28
TISH DEBUG gradient_percentage divided by base_price 5.953695789006536 to gradient_percentage divided by Predicted PSF start: 6.38989311253661
Gradient for KLANG LANE: gradient_per_year 33.3733, gradient_percentage 5.95%
Predicted PSF at start: 522.28, at end: 680.81, from 2019-03 to 2024-01, over 57.0 months. 60 data points
Saved: /content/drive/My Drive/1 Projects/ML Sing Property Prices/Python-generated Graphs/KLANG_LANE_2002_Price_Trend.html
